<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/pruebas_LuisIrigoyen/Modelo_14_NoisyLinear_V3_proyecto_final_APR_V01_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

- Igual que 12, cambio paramétro: dqn.compile(optimizer=Adam(learning_rate=0.0001)
- luego se modificó a dqn.compile(optimizer=Adam(learning_rate=1e-4)
- se cambió policy = BoltzmannQPolicy() a policy = EpsGreedyQPolicy(eps=0.0)

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/MODELO 14/"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

## 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/MODELO 14/  exists.

Colab: Changing directory to  /content/gdrive/My Drive/MODELO 14/
/content/gdrive/My Drive/MODELO 14
Archivos en el directorio: 
['logs', 'checkpoints', 'memory_noisy.pkl']


## 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-eolfz9cv
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-eolfz9cv
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [ ]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [ ]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        # Convertir a float y escalar
        # Linea Agregada
        if isinstance(batch, list):
          batch = np.array(batch)

        batch = batch.astype('float32') / 255.

        # 👇 Elimina el canal innecesario si existe (forma (B, 4, 84, 84, 1))
        if batch.ndim == 5 and batch.shape[-1] == 1:
            batch = np.squeeze(batch, axis=-1)

        return batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## 1. EJFM Implementación de la red neuronal

In [ ]:
import tensorflow as tf

class NoisyDense(tf.keras.layers.Layer):
    def __init__(self, units, sigma_init=0.017, **kwargs):
        super(NoisyDense, self).__init__(**kwargs)
        self.units = units
        self.sigma_init = sigma_init

    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        self.mu_W = self.add_weight("mu_W", shape=(input_dim, self.units),
                                    initializer='random_uniform', trainable=True)
        self.sigma_W = self.add_weight("sigma_W", shape=(input_dim, self.units),
                                       initializer=tf.keras.initializers.Constant(self.sigma_init), trainable=True)
        self.mu_b = self.add_weight("mu_b", shape=(self.units,),
                                    initializer='zeros', trainable=True)
        self.sigma_b = self.add_weight("sigma_b", shape=(self.units,),
                                       initializer=tf.keras.initializers.Constant(self.sigma_init), trainable=True)

    def call(self, x):
        epsilon_W = tf.random.normal((x.shape[-1], self.units))
        epsilon_b = tf.random.normal((self.units,))
        W = self.mu_W + self.sigma_W * epsilon_W
        b = self.mu_b + self.sigma_b * epsilon_b
        return tf.matmul(x, W) + b

    def get_config(self):
        config = super(NoisyDense, self).get_config()
        config.update({
            "units": self.units,
            "sigma_init": self.sigma_init
        })
        return config


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Permute, Convolution2D, Flatten, Activation

input_layer = Input(shape=(WINDOW_LENGTH,) + INPUT_SHAPE)
perm = Permute((2, 3, 1))(input_layer)  # Canal al final
conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(perm)
conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
conv3 = Convolution2D(64, (3, 3), strides=(1, 1), activation='relu')(conv2)
flat = Flatten()(conv3)

noisy1 = NoisyDense(512)(flat)
noisy1 = Activation('relu')(noisy1)
output_layer = NoisyDense(nb_actions)(noisy1)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4, 84, 84)]       0         
                                                                 
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 noisy_dense (NoisyDense)    (None, 512)               321228

In [ ]:
# Registrar la capa personalizada (una sola vez en todo el código)
from tensorflow.keras.utils import get_custom_objects
get_custom_objects().update({'NoisyDense': NoisyDense})

## 2. EJFM Implementación de la solución DQN

### Ejecución Inicial



In [ ]:
####### Solución DQN #######

import os
import pickle

# INTENTAR CARGAR LA MEMORIA
memory_file = 'memory_noisy.pkl'
if os.path.exists(memory_file):
    print("Cargando memoria desde archivo...")
    with open(memory_file, 'rb') as f:
        memory = pickle.load(f)
else:
    print("Creando nueva memoria...")
    memory = SequentialMemory(limit=1000000, window_length=4)

# Crear política como de costumbre
#policy = BoltzmannQPolicy()
policy = EpsGreedyQPolicy(eps=0.0)

# Crear agente
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=AtariProcessor(), nb_steps_warmup=50000, gamma=0.99,
               target_model_update=10000, train_interval=4, delta_clip=5)

# Compilar agente
#dqn.compile(optimizer='adam', metrics=['mae'])
#dqn.compile(optimizer=Adam(learning_rate=0.0001), metrics=['mae'])

from tensorflow.keras.optimizers.legacy import Adam     # ← incluye get_updates
# Compilar agente
dqn.compile(optimizer=Adam(learning_rate=1e-4), metrics=['mae'])

Cargando memoria desde archivo...


In [ ]:
####### Crear directorios  logs y checkpoints #######

import os
log_dir = os.path.join(drive_root, "logs")
checkpoint_dir = os.path.join(drive_root, "checkpoints")
os.makedirs(log_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Función para restaurar pesos más adelante
import re

def load_latest_weights(agent, checkpoint_dir, prefix="dqn_noisy_weights_"):
    """
    Busca el checkpoint más reciente, carga los pesos y retorna info útil.

    Parámetros:
        agent: el agente DQN ya creado y compilado.
        checkpoint_dir: carpeta con los archivos de pesos.
        prefix: prefijo de los archivos de pesos.

    Retorna:
        (ruta_archivo, pasos_entrenados) o (None, 0) si no se encuentra nada.
    """
    files = [f for f in os.listdir(checkpoint_dir) if f.endswith(".h5")]
    pattern = re.compile(rf"{re.escape(prefix)}(\d+)\.h5")

    candidates = []
    for f in files:
        match = pattern.match(f)
        if match:
            step = int(match.group(1))
            candidates.append((step, f))

    if not candidates:
        print("⚠️ No se encontraron checkpoints en:", checkpoint_dir)
        return None, 0

    # Escoger el de mayor número de pasos
    latest_step, latest_file = max(candidates, key=lambda x: x[0])
    latest_path = os.path.join(checkpoint_dir, latest_file)

    print(f"✅ Cargando pesos desde: {latest_path} (pasos entrenados: {latest_step})")
    agent.load_weights(latest_path)
    return latest_path, latest_step


In [ ]:
# Cargar pesos anteriores si existen
_, trained_steps = load_latest_weights(dqn, checkpoint_dir)

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_900000.h5 (pasos entrenados: 900000)


### Primera **Ejecución**

In [ ]:
# Cargar último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)

# Entrenar agente
dqn.fit(env, nb_steps=100000, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales en formato .h5 (compatible con load_weights)
dqn.save_weights(os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5"), overwrite=True)


✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_900000.h5 (pasos entrenados: 900000)
Training for 100000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   520/100000: episode: 1, duration: 5.540s, episode steps: 520, steps per second:  94, episode reward: 14.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 2.308 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1530/100000: episode: 2, duration: 10.808s, episode steps: 1010, steps per second:  93, episode reward: 28.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 2.081 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3034/100000: episode: 3, duration: 16.236s, episode steps: 1504, steps per second:  93, episode reward: 35.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 2.041 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3696/100000: episode: 4, duration: 7.761s, episode steps: 662, steps per second:  85, episode reward: 15.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 2.269 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  4351/100000: episode: 5, duration: 6.897s, episode steps: 655, steps per second:  95, episode reward: 17.000, mean reward:  0.026 

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 50541/100000: episode: 60, duration: 26.127s, episode steps: 663, steps per second:  25, episode reward: 11.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.385 [0.000, 5.000],  loss: 0.008427, mae: 0.856112, mean_q: 1.041659
 51934/100000: episode: 61, duration: 61.098s, episode steps: 1393, steps per second:  23, episode reward: 22.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.169 [0.000, 5.000],  loss: 0.008610, mae: 0.847622, mean_q: 1.030363
 52569/100000: episode: 62, duration: 28.179s, episode steps: 635, steps per second:  23, episode reward: 13.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.808 [0.000, 5.000],  loss: 0.007872, mae: 0.842190, mean_q: 1.024394
 53193/100000: episode: 63, duration: 27.576s, episode steps: 624, steps per second:  23, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.154 [0.000, 5.000],  loss: 0.008859, mae: 0.855168, mean_q: 1.039259
 53946/100000: episode: 64, duration: 33.343s, episode 

In [ ]:
# GUARDAR LA MEMORIA DE EXPERIENCIA
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1000000.h5 (pasos entrenados: 1000000)
Testing for 10 episodes ...
Episode 1: reward: 14.000, steps: 777
Episode 2: reward: 27.000, steps: 1068
Episode 3: reward: 15.000, steps: 750
Episode 4: reward: 12.000, steps: 484
Episode 5: reward: 10.000, steps: 601
Episode 6: reward: 15.000, steps: 666
Episode 7: reward: 16.000, steps: 731
Episode 8: reward: 20.000, steps: 1022
Episode 9: reward: 12.000, steps: 559
Episode 10: reward: 22.000, steps: 924


### Parte 2

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1000000.h5 (pasos entrenados: 1000000)
Training for 100000 steps ...
   623/100000: episode: 1, duration: 7.017s, episode steps: 623, steps per second:  89, episode reward: 10.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.681 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1278/100000: episode: 2, duration: 7.013s, episode steps: 655, steps per second:  93, episode reward: 13.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.724 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1992/100000: episode: 3, duration: 7.925s, episode steps: 714, steps per second:  90, episode reward: 15.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 3.099 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2908/100000: episode: 4, duration: 10.093s, episode steps: 916, steps per second:  91, episode reward: 22.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 3.198 [0.000, 5.000],  loss: -

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1100000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1100000.h5 (pasos entrenados: 1100000)
Testing for 10 episodes ...
Episode 1: reward: 22.000, steps: 1099
Episode 2: reward: 14.000, steps: 732
Episode 3: reward: 14.000, steps: 688
Episode 4: reward: 16.000, steps: 888
Episode 5: reward: 10.000, steps: 654
Episode 6: reward: 12.000, steps: 676
Episode 7: reward: 11.000, steps: 612
Episode 8: reward: 11.000, steps: 678
Episode 9: reward: 11.000, steps: 562
Episode 10: reward: 10.000, steps: 526


Parte 3

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1100000.h5 (pasos entrenados: 1100000)
Training for 100000 steps ...
  1122/100000: episode: 1, duration: 12.463s, episode steps: 1122, steps per second:  90, episode reward: 28.000, mean reward:  0.025 [ 0.000,  1.000], mean action: 1.758 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1751/100000: episode: 2, duration: 7.016s, episode steps: 629, steps per second:  90, episode reward: 10.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 1.808 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2403/100000: episode: 3, duration: 7.126s, episode steps: 652, steps per second:  91, episode reward:  9.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 1.913 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3778/100000: episode: 4, duration: 15.175s, episode steps: 1375, steps per second:  91, episode reward: 26.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 1.731 [0.000, 5.000],  loss

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1200000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1200000.h5 (pasos entrenados: 1200000)
Testing for 10 episodes ...
Episode 1: reward: 10.000, steps: 593
Episode 2: reward: 22.000, steps: 1022
Episode 3: reward: 21.000, steps: 1032
Episode 4: reward: 9.000, steps: 641
Episode 5: reward: 19.000, steps: 1056
Episode 6: reward: 8.000, steps: 640
Episode 7: reward: 11.000, steps: 541
Episode 8: reward: 7.000, steps: 482
Episode 9: reward: 21.000, steps: 871
Episode 10: reward: 7.000, steps: 623


Parte 4

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1200000.h5 (pasos entrenados: 1200000)
Training for 100000 steps ...
   884/100000: episode: 1, duration: 9.702s, episode steps: 884, steps per second:  91, episode reward: 17.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 3.191 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1893/100000: episode: 2, duration: 10.936s, episode steps: 1009, steps per second:  92, episode reward: 20.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 3.139 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3023/100000: episode: 3, duration: 12.162s, episode steps: 1130, steps per second:  93, episode reward: 27.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 3.092 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3676/100000: episode: 4, duration: 6.957s, episode steps: 653, steps per second:  94, episode reward:  8.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 3.029 [0.000, 5.000],  loss

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1300000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1300000.h5 (pasos entrenados: 1300000)
Testing for 10 episodes ...
Episode 1: reward: 12.000, steps: 651
Episode 2: reward: 29.000, steps: 1225
Episode 3: reward: 13.000, steps: 617
Episode 4: reward: 14.000, steps: 717
Episode 5: reward: 14.000, steps: 687
Episode 6: reward: 8.000, steps: 538
Episode 7: reward: 9.000, steps: 568
Episode 8: reward: 6.000, steps: 479
Episode 9: reward: 11.000, steps: 536
Episode 10: reward: 23.000, steps: 1078


Parte 5

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1300000.h5 (pasos entrenados: 1300000)
Training for 100000 steps ...
   622/100000: episode: 1, duration: 6.955s, episode steps: 622, steps per second:  89, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 2.543 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1525/100000: episode: 2, duration: 9.920s, episode steps: 903, steps per second:  91, episode reward: 21.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 2.288 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2250/100000: episode: 3, duration: 7.882s, episode steps: 725, steps per second:  92, episode reward: 15.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 2.588 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3102/100000: episode: 4, duration: 9.504s, episode steps: 852, steps per second:  90, episode reward: 24.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 2.221 [0.000, 5.000],  loss: --

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1400000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1400000.h5 (pasos entrenados: 1400000)
Testing for 10 episodes ...
Episode 1: reward: 8.000, steps: 521
Episode 2: reward: 8.000, steps: 524
Episode 3: reward: 11.000, steps: 534
Episode 4: reward: 8.000, steps: 549
Episode 5: reward: 18.000, steps: 733
Episode 6: reward: 11.000, steps: 561
Episode 7: reward: 8.000, steps: 462
Episode 8: reward: 29.000, steps: 1161
Episode 9: reward: 10.000, steps: 560
Episode 10: reward: 17.000, steps: 668


Parte 6

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1400000.h5 (pasos entrenados: 1400000)
Training for 100000 steps ...
   648/100000: episode: 1, duration: 7.311s, episode steps: 648, steps per second:  89, episode reward: 15.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 2.086 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1371/100000: episode: 2, duration: 7.943s, episode steps: 723, steps per second:  91, episode reward: 16.000, mean reward:  0.022 [ 0.000,  1.000], mean action: 2.165 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1890/100000: episode: 3, duration: 5.680s, episode steps: 519, steps per second:  91, episode reward: 11.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 2.027 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2258/100000: episode: 4, duration: 3.955s, episode steps: 368, steps per second:  93, episode reward:  5.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.171 [0.000, 5.000],  loss: --

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1500000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1500000.h5 (pasos entrenados: 1500000)
Testing for 10 episodes ...
Episode 1: reward: 28.000, steps: 1213
Episode 2: reward: 35.000, steps: 1513
Episode 3: reward: 16.000, steps: 719
Episode 4: reward: 12.000, steps: 588
Episode 5: reward: 19.000, steps: 820
Episode 6: reward: 25.000, steps: 925
Episode 7: reward: 29.000, steps: 1524
Episode 8: reward: 16.000, steps: 742
Episode 9: reward: 18.000, steps: 886
Episode 10: reward: 25.000, steps: 1118


Parte 7

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1500000.h5 (pasos entrenados: 1500000)
Training for 100000 steps ...
  1082/100000: episode: 1, duration: 11.868s, episode steps: 1082, steps per second:  91, episode reward: 24.000, mean reward:  0.022 [ 0.000,  1.000], mean action: 1.689 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1688/100000: episode: 2, duration: 6.886s, episode steps: 606, steps per second:  88, episode reward: 11.000, mean reward:  0.018 [ 0.000,  1.000], mean action: 1.926 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2599/100000: episode: 3, duration: 9.938s, episode steps: 911, steps per second:  92, episode reward: 18.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 1.708 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3377/100000: episode: 4, duration: 8.628s, episode steps: 778, steps per second:  90, episode reward: 21.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 1.684 [0.000, 5.000],  loss: 

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1600000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1600000.h5 (pasos entrenados: 1600000)
Testing for 10 episodes ...
Episode 1: reward: 10.000, steps: 487
Episode 2: reward: 18.000, steps: 785
Episode 3: reward: 17.000, steps: 599
Episode 4: reward: 20.000, steps: 839
Episode 5: reward: 34.000, steps: 1117
Episode 6: reward: 20.000, steps: 838
Episode 7: reward: 26.000, steps: 938
Episode 8: reward: 26.000, steps: 1090
Episode 9: reward: 25.000, steps: 962
Episode 10: reward: 16.000, steps: 661


Parte 8

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1600000.h5 (pasos entrenados: 1600000)
Training for 100000 steps ...
  1143/100000: episode: 1, duration: 12.265s, episode steps: 1143, steps per second:  93, episode reward: 31.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 2.224 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2002/100000: episode: 2, duration: 9.247s, episode steps: 859, steps per second:  93, episode reward: 23.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 2.242 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2831/100000: episode: 3, duration: 9.005s, episode steps: 829, steps per second:  92, episode reward: 20.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 1.948 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  4278/100000: episode: 4, duration: 15.354s, episode steps: 1447, steps per second:  94, episode reward: 41.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 1.758 [0.000, 5.000],  loss

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1700000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1700000.h5 (pasos entrenados: 1700000)
Testing for 10 episodes ...
Episode 1: reward: 37.000, steps: 1422
Episode 2: reward: 19.000, steps: 672
Episode 3: reward: 7.000, steps: 406
Episode 4: reward: 22.000, steps: 714
Episode 5: reward: 17.000, steps: 688
Episode 6: reward: 26.000, steps: 1066
Episode 7: reward: 30.000, steps: 1011
Episode 8: reward: 11.000, steps: 699
Episode 9: reward: 27.000, steps: 1134
Episode 10: reward: 32.000, steps: 1337


Parte 9

In [ ]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1700000.h5 (pasos entrenados: 1700000)
Training for 100000 steps ...
   951/100000: episode: 1, duration: 10.163s, episode steps: 951, steps per second:  94, episode reward: 23.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 2.592 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1584/100000: episode: 2, duration: 6.852s, episode steps: 633, steps per second:  92, episode reward: 17.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 2.319 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2619/100000: episode: 3, duration: 11.234s, episode steps: 1035, steps per second:  92, episode reward: 28.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 2.329 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3379/100000: episode: 4, duration: 8.274s, episode steps: 760, steps per second:  92, episode reward: 18.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 2.449 [0.000, 5.000],  loss:

In [ ]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1800000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [ ]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1800000.h5 (pasos entrenados: 1800000)
Testing for 10 episodes ...
Episode 1: reward: 26.000, steps: 997
Episode 2: reward: 16.000, steps: 665
Episode 3: reward: 11.000, steps: 568
Episode 4: reward: 15.000, steps: 550
Episode 5: reward: 25.000, steps: 770
Episode 6: reward: 26.000, steps: 949
Episode 7: reward: 30.000, steps: 1111
Episode 8: reward: 20.000, steps: 789
Episode 9: reward: 18.000, steps: 740
Episode 10: reward: 13.000, steps: 670


Parte 10

In [50]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1800000.h5 (pasos entrenados: 1800000)
Training for 100000 steps ...
   957/100000: episode: 1, duration: 10.726s, episode steps: 957, steps per second:  89, episode reward: 26.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 2.447 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1446/100000: episode: 2, duration: 5.543s, episode steps: 489, steps per second:  88, episode reward: 10.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 3.025 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2326/100000: episode: 3, duration: 9.764s, episode steps: 880, steps per second:  90, episode reward: 26.000, mean reward:  0.030 [ 0.000,  1.000], mean action: 2.431 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3163/100000: episode: 4, duration: 9.306s, episode steps: 837, steps per second:  90, episode reward: 20.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 2.265 [0.000, 5.000],  loss: -

In [51]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 1900000 pasos completado y guardado.


In [ ]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

In [53]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1900000.h5 (pasos entrenados: 1900000)
Testing for 10 episodes ...
Episode 1: reward: 13.000, steps: 509
Episode 2: reward: 17.000, steps: 689
Episode 3: reward: 13.000, steps: 499
Episode 4: reward: 27.000, steps: 897
Episode 5: reward: 12.000, steps: 579
Episode 6: reward: 15.000, steps: 752
Episode 7: reward: 13.000, steps: 688
Episode 8: reward: 30.000, steps: 1257
Episode 9: reward: 16.000, steps: 579
Episode 10: reward: 25.000, steps: 863


Parte 11

In [54]:
# Cargar pesos anteriores si existen
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.load_weights(latest_path)
else:
    trained_steps = 0

# Definir pasos a entrenar
steps_to_train = 100000
total_steps = trained_steps + steps_to_train

# Crear paths y callbacks dinámicos
log_path = os.path.join(log_dir, f"dqn_noisy_log_{trained_steps}_{total_steps}.json")
checkpoint_path = os.path.join(checkpoint_dir, f"dqn_noisy_weights_{total_steps}.h5")

logger_callback = FileLogger(log_path, interval=100)
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_path, interval=10000)

# Entrenar agente
dqn.fit(env, nb_steps=steps_to_train, visualize=False, verbose=2,
        callbacks=[checkpoint_callback, logger_callback])

# Guardar pesos finales
dqn.save_weights(checkpoint_path, overwrite=True)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_1900000.h5 (pasos entrenados: 1900000)
Training for 100000 steps ...
   693/100000: episode: 1, duration: 7.443s, episode steps: 693, steps per second:  93, episode reward: 19.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 1.749 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1547/100000: episode: 2, duration: 8.892s, episode steps: 854, steps per second:  96, episode reward: 21.000, mean reward:  0.025 [ 0.000,  1.000], mean action: 1.957 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2459/100000: episode: 3, duration: 9.466s, episode steps: 912, steps per second:  96, episode reward: 25.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 1.830 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3302/100000: episode: 4, duration: 8.963s, episode steps: 843, steps per second:  94, episode reward: 21.000, mean reward:  0.025 [ 0.000,  1.000], mean action: 2.006 [0.000, 5.000],  loss: --

In [55]:
# Guardar la memoria de experiencia
import pickle
with open('memory_noisy.pkl', 'wb') as f:
    pickle.dump(memory, f)

print(f"Entrenamiento hasta {total_steps} pasos completado y guardado.")

Entrenamiento hasta 2000000 pasos completado y guardado.


In [56]:
import time

print("Esperando 60 segundos...")
time.sleep(60)
print("¡Tiempo de espera terminado!")

Esperando 60 segundos...
¡Tiempo de espera terminado!


In [57]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=10, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_2000000.h5 (pasos entrenados: 2000000)
Testing for 10 episodes ...
Episode 1: reward: 25.000, steps: 920
Episode 2: reward: 25.000, steps: 767
Episode 3: reward: 21.000, steps: 688
Episode 4: reward: 33.000, steps: 1188
Episode 5: reward: 29.000, steps: 940
Episode 6: reward: 33.000, steps: 1274
Episode 7: reward: 20.000, steps: 815
Episode 8: reward: 30.000, steps: 1009
Episode 9: reward: 22.000, steps: 816
Episode 10: reward: 15.000, steps: 670


test final

In [58]:
# Testear desde último checkpoint
latest_path, trained_steps = load_latest_weights(dqn, checkpoint_dir)
if latest_path:
    dqn.test(env, nb_episodes=50, visualize=False)

✅ Cargando pesos desde: /content/gdrive/My Drive/MODELO 14/checkpoints/dqn_noisy_weights_2000000.h5 (pasos entrenados: 2000000)
Testing for 50 episodes ...
Episode 1: reward: 25.000, steps: 850
Episode 2: reward: 22.000, steps: 894
Episode 3: reward: 23.000, steps: 889
Episode 4: reward: 31.000, steps: 1294
Episode 5: reward: 26.000, steps: 945
Episode 6: reward: 29.000, steps: 1144
Episode 7: reward: 36.000, steps: 1234
Episode 8: reward: 23.000, steps: 971
Episode 9: reward: 17.000, steps: 731
Episode 10: reward: 24.000, steps: 891
Episode 11: reward: 23.000, steps: 762
Episode 12: reward: 8.000, steps: 429
Episode 13: reward: 16.000, steps: 667
Episode 14: reward: 24.000, steps: 838
Episode 15: reward: 20.000, steps: 754
Episode 16: reward: 20.000, steps: 904
Episode 17: reward: 23.000, steps: 1024
Episode 18: reward: 15.000, steps: 743
Episode 19: reward: 24.000, steps: 837
Episode 20: reward: 25.000, steps: 919
Episode 21: reward: 24.000, steps: 1084
Episode 22: reward: 17.000, st